# Transfer Learning – Binary Classification (Colab)
Execute as células em ordem. Você pode usar `cats_vs_dogs` (TFDS) ou seu próprio dataset.

In [ ]:

#@title Configuração rápida
USE_TFDS = True  #@param {type:"boolean"}
IMG_SIZE = 224   #@param {type:"integer"}
BATCH_SIZE = 32  #@param {type:"integer"}
EPOCHS = 3       #@param {type:"integer"}

# Se USE_TFDS=False, defina o caminho da pasta no seu Google Drive com 2 classes (class_a, class_b)
CUSTOM_DATA_DIR = "/content/drive/MyDrive/dataset_binary"  #@param {type:"string"}


In [ ]:

#@title Instalação/Imports
import os, math, pathlib
import tensorflow as tf
print("TensorFlow:", tf.__version__)

try:
    import tensorflow_datasets as tfds
    TFDS_OK = True
except Exception as e:
    TFDS_OK = False
    print("Aviso: tensorflow_datasets não disponível:", e)


In [ ]:

#@title Carregar dados (TFDS ou pasta no Drive)
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory

if USE_TFDS:
    assert TFDS_OK, "tensorflow_datasets não está disponível neste ambiente."
    (ds_train, ds_test), ds_info = tfds.load(
        'cats_vs_dogs',
        split=['train[:80%]', 'train[80%:]'],
        as_supervised=True,
        with_info=True,
    )

    CLASS_NAMES = ['cat', 'dog']
    num_train = ds_info.splits['train'].num_examples * 8 // 10
    num_val = ds_info.splits['train'].num_examples - num_train

    def preprocess(image, label):
        image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
        image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
        return image, tf.cast(label, tf.float32)

    AUTOTUNE = tf.data.AUTOTUNE
    train_ds = ds_train.map(preprocess, num_parallel_calls=AUTOTUNE).shuffle(2048).batch(BATCH_SIZE).prefetch(AUTOTUNE)
    val_ds   = ds_test.map(preprocess,  num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)

else:
    # Montar Google Drive se necessário
    from google.colab import drive
    drive.mount('/content/drive')

    data_dir = pathlib.Path(CUSTOM_DATA_DIR)
    assert data_dir.exists(), f"Pasta não encontrada: {CUSTOM_DATA_DIR}"

    train_ds = image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="training",
        seed=1337,
        image_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE
    )
    val_ds = image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="validation",
        seed=1337,
        image_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE
    )
    CLASS_NAMES = train_ds.class_names

    AUTOTUNE = tf.data.AUTOTUNE
    # Normalização para MobileNetV2
    def mobilenet_preprocess(image, label):
        image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
        return image, tf.cast(label, tf.float32)

    train_ds = train_ds.map(mobilenet_preprocess, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
    val_ds   = val_ds.map(mobilenet_preprocess,   num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

print("Classes:", CLASS_NAMES)


In [ ]:

#@title Construir modelo (MobileNetV2 congelada + cabeça binária)
from tensorflow.keras import layers, models

base = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base.trainable = False  # congela a base

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = inputs
# a imagem já foi preprocessada; se quiser, pode incluir data augmentation leve:
# x = layers.RandomFlip("horizontal")(x)
# x = layers.RandomRotation(0.05)(x)

x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)  # binário

model = models.Model(inputs, outputs)
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()


In [ ]:

#@title Treinar (few epochs)
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)


In [ ]:

#@title Avaliar e salvar modelo
val_loss, val_acc = model.evaluate(val_ds, verbose=0)
print(f"Acurácia de validação: {val_acc:.4f}")

# Salvar modelo
model_path = "/content/model_mobilenetv2_binary.keras"
model.save(model_path)
print("Modelo salvo em:", model_path)


In [ ]:

#@title (Opcional) Gráfico de perda e acurácia
import matplotlib.pyplot as plt

acc = history.history.get('accuracy', [])
val_acc = history.history.get('val_accuracy', [])
loss = history.history.get('loss', [])
val_loss = history.history.get('val_loss', [])

plt.figure()
plt.plot(acc, label='acc')
plt.plot(val_acc, label='val_acc')
plt.title('Acurácia')
plt.xlabel('época'); plt.ylabel('acc')
plt.legend()
plt.show()

plt.figure()
plt.plot(loss, label='loss')
plt.plot(val_loss, label='val_loss')
plt.title('Perda')
plt.xlabel('época'); plt.ylabel('loss')
plt.legend()
plt.show()
